In [1]:
import os, numpy as np
import pandas as pd

os.environ["KERAS_BACKEND"] = "tensorflow"

import keras 
from sklearn import datasets

from sklearn.model_selection import train_test_split

keras.backend.set_floatx('float32')

from keras.utils import np_utils
from keras.constraints import max_norm


def set_reproducible():
    import tensorflow as tf
    import random as rn
    import os
    os.environ['PYTHONHASHSEED'] = '1960'
    rn.seed(1960)
    np.random.seed(1960)
    session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

    from keras import backend as K
    tf.set_random_seed(1960)

    sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    # K.set_session(sess)

set_reproducible()


Using TensorFlow backend.


# Build a Keras Model

In [2]:
NC = 13

def create_model():
    from keras.models import Sequential
    from keras.layers import Dense, Activation, LSTM
    from keras.constraints import max_norm

    model = Sequential()
    model.add(LSTM(1, input_shape=(1, NC), batch_size=1 , 
                   activation='relu', recurrent_activation = 'hard_sigmoid', 
                   stateful=True,
                   kernel_constraint=max_norm(2.),
                   recurrent_constraint=max_norm(2.),
                   kernel_initializer = keras.initializers.Orthogonal(gain=0.1),
                   recurrent_initializer = keras.initializers.Zeros()))
    model.add(Dense(1, activation=None))
    adam_optimizer = keras.optimizers.Adam(lr=0.01, clipnorm=1.)
    model.compile(loss='mse', optimizer=adam_optimizer)
    return model


In [3]:
boston  = datasets.load_boston()
boston.data = (boston.data - boston.data.mean(axis=0)) / boston.data.std(axis=0)
boston.target = (boston.target - boston.target.mean(axis=0)) / boston.target.std(axis=0)

boston.data = boston.data[:,0:NC]
# boston.target = boston.data[:,1]

train_X, test_X, train_y, test_y = train_test_split(boston.data, boston.target, train_size=0.8, test_size=0.2, random_state=1960)
train_X = train_X.reshape(train_X.shape[0] , 1, train_X.shape[1])
test_X = test_X.reshape(test_X.shape[0] , 1, test_X.shape[1])



In [4]:
# train_X

In [5]:
#boston.target

In [6]:

from keras.wrappers.scikit_learn import KerasRegressor

clf = KerasRegressor(build_fn=create_model, batch_size=1, epochs=12, verbose=2)

print(train_X.shape , train_y.shape, train_X.dtype , train_y.dtype)
clf.fit(train_X, train_y)

(404, 1, 13) (404,) float64 float64
Epoch 1/12
 - 5s - loss: 0.4529
Epoch 2/12
 - 3s - loss: 0.2724
Epoch 3/12
 - 3s - loss: 0.2332
Epoch 4/12
 - 3s - loss: 0.2074
Epoch 5/12
 - 3s - loss: 0.2301
Epoch 6/12
 - 3s - loss: 0.2079
Epoch 7/12
 - 2s - loss: 0.2349
Epoch 8/12
 - 3s - loss: 0.2193
Epoch 9/12
 - 2s - loss: 0.2211
Epoch 10/12
 - 1s - loss: 0.2088
Epoch 11/12
 - 1s - loss: 0.2098
Epoch 12/12
 - 1s - loss: 0.2016


In [7]:
print(test_X.shape)
preds = clf.predict(test_X)
clf.model.reset_states()
print(preds)


(102, 1, 13)
[-0.22588879 -0.51580876 -0.2410878  -0.54559267  0.22782338 -0.5725206
 -0.6540518  -0.74347055 -1.0372039  -1.0372039  -0.84205705 -0.7621927
 -0.19718373 -1.0372039   1.1956807  -0.8132057   0.774145   -0.25460047
 -1.0372039   0.27320802 -0.07806408  1.9324733  -1.0372039   0.05064917
  0.5851741  -0.382416   -0.29469872 -1.0372039  -0.1506483  -1.0372039
  0.15698183 -1.0163854  -1.0372039   0.97233784 -0.06239009 -0.20518976
  0.2121588  -1.0372039  -0.90382165  0.28622735 -0.7210716  -0.27013743
 -0.33481997 -1.0372039  -0.21277684 -0.2852546   0.06916034 -0.11272919
 -1.0372039  -0.83141637 -0.48814106 -0.81684303 -1.0372039   1.3147925
 -0.5674504  -0.9928935   0.80365276 -0.28945583 -0.40703338 -0.3676477
 -0.96918577  1.0488528  -0.47345603 -0.12919623 -1.0372039   1.2674214
 -0.8158749  -0.83365154 -1.0372039  -0.37442487 -0.04633254  1.8504056
 -0.4345302  -0.36566675  0.01433587 -1.0372039   1.3804206   0.28230584
 -1.0228282   0.5215901  -0.74603975  0.04061

In [8]:
pd.DataFrame(boston.data).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12
count,506.000000,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02
mean,0.000000,6.319056e-17,-3.145486e-15,-2.106352e-17,2.752300e-15,-1.150770e-14,-1.137430e-15,7.582867e-16,5.616939e-17,5.616939e-17,-1.022283e-14,8.593916e-15,-5.897786e-16
std,1.000990,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00
min,-0.417713,-4.877224e-01,-1.557842e+00,-2.725986e-01,-1.465882e+00,-3.880249e+00,-2.335437e+00,-1.267069e+00,-9.828429e-01,-1.313990e+00,-2.707379e+00,-3.907193e+00,-1.531127e+00
25%,-0.408896,-4.877224e-01,-8.676906e-01,-2.725986e-01,-9.130288e-01,-5.686303e-01,-8.374480e-01,-8.056878e-01,-6.379618e-01,-7.675760e-01,-4.880391e-01,2.050715e-01,-7.994200e-01
50%,-0.388582,-4.877224e-01,-2.110985e-01,-2.725986e-01,-1.442174e-01,-1.084655e-01,3.173816e-01,-2.793234e-01,-5.230014e-01,-4.646726e-01,2.748590e-01,3.811865e-01,-1.812536e-01
75%,0.006248,4.877224e-02,1.015999e+00,-2.725986e-01,5.986790e-01,4.827678e-01,9.067981e-01,6.623709e-01,1.661245e+00,1.530926e+00,8.065758e-01,4.336510e-01,6.030188e-01
max,9.941735,3.804234e+00,2.422565e+00,3.668398e+00,2.732346e+00,3.555044e+00,1.117494e+00,3.960518e+00,1.661245e+00,1.798194e+00,1.638828e+00,4.410519e-01,3.548771e+00


# Generate SQL Code from the Model

In [9]:
import json, requests, base64, dill as pickle, sys



sys.setrecursionlimit(200000)
pickle.settings['recurse'] = False

# no luck for the web service... pickling feature of tensorflow and/or keras objects seems not to be a priority.
# there is a lot of github issues in the two projects when I search for pickle keyword!!!.

def test_ws_sql_gen(pickle_data):
    WS_URL="http://localhost:1888/model"
    b64_data = base64.b64encode(pickle_data).decode('utf-8')
    data={"Name":"model1", "PickleData":b64_data , "SQLDialect":"sqlite"}
    r = requests.post(WS_URL, json=data)
    print(r.__dict__)
    content = r.json()
    # print(content)
    lSQL = content["model"]["SQLGenrationResult"][0]["SQL"]
    return lSQL;



def test_sql_gen(keras_regressor , metadata):
    import sklearn2sql.PyCodeGenerator as codegen
    cg1 = codegen.cAbstractCodeGenerator();
    cg1.mOptions.mActivateMaterialization = False
    lSQL = cg1.generateCodeWithMetadata(clf, metadata, dsn = None, dialect = "sqlite");
    return lSQL[1]


In [10]:
# commented .. see above
# pickle_data = pickle.dumps(clf)
# lSQL = test_ws_sql_gen(pickle_data)
# print(lSQL[0:2000])

In [11]:
lMetaData = {}
lMetaData['features'] = list(boston.feature_names)[0:NC];
lMetaData["targets"] = ['TGT']
lMetaData['primary_key'] = 'KEY'
lMetaData['table'] = 'boston'

lSQL = test_sql_gen(clf , lMetaData)


cGenerationWrapperFactory::createWrapper() <class 'keras.wrappers.scikit_learn.KerasRegressor'>
BACKEND_DIALECT sqlite
CREATING_DATABASE_BACKEND_DSN_DIALECT 1.2.7 None sqlite
KERAS_REG_GENERATE_EXPRESSION_START <class 'sqlalchemy.sql.selectable.Alias'> <class 'sklearn2sql.GenericModels.Keras.cSerializedKeras'>
{'mKerasData': <keras.wrappers.scikit_learn.KerasRegressor object at 0x7fae93da6898>, 'mFeatureNames': ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT'], 'mClasses': None, 'mMode': 'Regression'}
{'build_fn': <function create_model at 0x7fae93d9b7b8>, 'sk_params': {'batch_size': 1, 'epochs': 12, 'verbose': 2}, 'model': <keras.engine.sequential.Sequential object at 0x7fae93da6780>}
GENERATING_LAYER 0 lstm_1 <class 'keras.layers.recurrent.LSTM'> {'input_spec': [InputSpec(shape=(1, None, 13), ndim=3)], 'supports_masking': True, 'stateful': True, '_trainable_weights': [], '_non_trainable_weights': [], '_losses': [], '_updates': [(<tf.Var

In [12]:
print(lSQL)

WITH RECURSIVE keras_input AS 
(SELECT "ADS"."KEY" AS "KEY", "ADS"."CRIM" AS "CRIM", "ADS"."ZN" AS "ZN", "ADS"."INDUS" AS "INDUS", "ADS"."CHAS" AS "CHAS", "ADS"."NOX" AS "NOX", "ADS"."RM" AS "RM", "ADS"."AGE" AS "AGE", "ADS"."DIS" AS "DIS", "ADS"."RAD" AS "RAD", "ADS"."TAX" AS "TAX", "ADS"."PTRATIO" AS "PTRATIO", "ADS"."B" AS "B", "ADS"."LSTAT" AS "LSTAT" 
FROM boston AS "ADS"), 
keras_input_1 AS 
(SELECT keras_input."KEY" AS "KEY", keras_input."CRIM" AS "CRIM", keras_input."ZN" AS "ZN", keras_input."INDUS" AS "INDUS", keras_input."CHAS" AS "CHAS", keras_input."NOX" AS "NOX", keras_input."RM" AS "RM", keras_input."AGE" AS "AGE", keras_input."DIS" AS "DIS", keras_input."RAD" AS "RAD", keras_input."TAX" AS "TAX", keras_input."PTRATIO" AS "PTRATIO", keras_input."B" AS "B", keras_input."LSTAT" AS "LSTAT" 
FROM keras_input), 
keras_input_1_rn AS 
(SELECT (SELECT count(t."KEY") AS count_1 
FROM (SELECT keras_input_1."KEY" AS "KEY" 
FROM keras_input_1) AS t 
WHERE t."KEY" <= keras_input_1."KE

# Execute the SQL Code

In [13]:
# save the dataset in a database table


import sqlalchemy as sa

def null_protected_function_1_arg(f , x):
    if(x is None):
        return x;
    return f(x);

def null_protected_function_2_args(f , x , n):
    if(x is None):
        return x;
    if(n is None):
        return n;
    return f(x , n);


def declareAdditionalFunctionsForSQLite(conn):
    import numpy as np
    import math
    from scipy import stats

    my_exp = lambda x : null_protected_function_1_arg(np.exp, x);
    conn.connection.connection.create_function("exp", 1, my_exp) 
    my_log = lambda x : null_protected_function_1_arg(np.log, x);
    conn.connection.connection.create_function("ln", 1, my_log) 
    my_tanh = lambda x : null_protected_function_1_arg(np.tanh, x);
    conn.connection.connection.create_function("tanh", 1, my_tanh) 
    my_pow = lambda x, n : null_protected_function_2_args(math.pow, x , n);
    conn.connection.connection.create_function("power", 2, my_pow) 
    my_sign = lambda x : null_protected_function_1_arg(np.sign, x);
    conn.connection.connection.create_function("sign", 1, my_sign) 
    my_sqrt = lambda x : null_protected_function_1_arg(np.sqrt, x);
    conn.connection.connection.create_function("sqrt", 1, my_sqrt) 
    my_inv_norm_cdf = lambda x : null_protected_function_1_arg(stats.norm.ppf, x);
    conn.connection.connection.create_function("inv_norm_cdf", 1, my_inv_norm_cdf) 


lDSN = 'sqlite://'
# lDSN = "postgresql://db:db@localhost/db?port=5432"
# lDSN = "mysql://db:db@localhost/db"
engine = sa.create_engine(lDSN, echo=False)
conn = engine.connect()
declareAdditionalFunctionsForSQLite(conn)

lTable = pd.DataFrame(boston.data);
lTable.columns = lMetaData['features']
lTable['TGT'] = boston.target
lTable['KEY'] = range(boston.data.shape[0])
lTable.to_sql(lMetaData['table'] , conn,   if_exists='replace', index=False)

In [14]:
sql_output = pd.read_sql(lSQL , conn);
conn.close()

In [15]:
sql_output.sample(12, random_state=1960)

,KEY,Estimator
230,230,-0.287226
112,112,-0.434198
125,125,-0.225111
9,9,-0.423400
213,213,0.188965
109,109,-0.462561
127,127,-0.602170
244,244,-0.847406
406,406,-1.037204
490,490,-1.037204


# Keras Prediction

In [16]:
#clf.model.reset_states()
skl_output = pd.DataFrame()
skl_output_key = pd.DataFrame(list(range(boston.data.shape[0])), columns=['KEY']);
data = boston.data.reshape(boston.data.shape[0] , 1, boston.data.shape[1])
skl_out = clf.predict(data)
print(skl_out.shape)
skl_output_estimator = pd.DataFrame(skl_out, columns=['Estimator'])
skl_output['KEY'] = skl_output_key['KEY']
skl_output['Estimator'] = skl_output_estimator
skl_output.sample(12, random_state=1960)

(506,)


,KEY,Estimator
230,230,-0.287226
112,112,-0.434198
125,125,-0.225111
9,9,-0.423400
213,213,0.188965
109,109,-0.462561
127,127,-0.602170
244,244,-0.847406
406,406,-1.037204
490,490,-1.037204


# Comparing the SQL and Keras Predictions

In [17]:
sql_skl_join = skl_output.join(sql_output , how='left', on='KEY', lsuffix='_skl', rsuffix='_sql')
sql_skl_join['Error'] = sql_skl_join.Estimator_sql - sql_skl_join.Estimator_skl

In [18]:
sql_skl_join.head(12)

,KEY_skl,Estimator_skl,KEY_sql,Estimator_sql,Error
0,0,0.576092,0,0.576091,-7.470172e-08
1,1,-0.136534,1,-0.136534,2.519421e-09
2,2,0.931535,2,0.931535,-1.192708e-07
3,3,0.675004,3,0.675005,1.897668e-07
4,4,0.640478,4,0.640478,-9.786047e-08
5,5,0.014058,5,0.014058,4.477145e-08
6,6,-0.310616,6,-0.310616,-3.851279e-08
7,7,-0.571255,7,-0.571255,7.503791e-08
8,8,-1.037204,8,-1.037204,0.000000e+00
9,9,-0.423400,9,-0.423400,2.546686e-09


In [19]:
sql_skl_join.describe()

,KEY_skl,Estimator_skl,KEY_sql,Estimator_sql,Error
count,506.000000,506.000000,506.000000,506.000000,5.060000e+02
mean,252.500000,-0.172709,252.500000,-0.172709,-8.020228e-09
std,146.213884,0.741151,146.213884,0.741151,1.002741e-07
min,0.000000,-1.037204,0.000000,-1.037204,-5.474613e-07
25%,126.250000,-0.760493,126.250000,-0.760493,-4.092312e-08
50%,252.500000,-0.297832,252.500000,-0.297832,0.000000e+00
75%,378.750000,0.179918,378.750000,0.179918,3.672368e-08
max,505.000000,2.490843,505.000000,2.490843,6.210996e-07
